# Lab 6: A LSTM for Part-of-Speech Tagging



## Part-of-Speech Tagging


In this section, we will use an LSTM to get part of speech tags.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocabulary. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.





In [ ]:
%matplotlib inline

## Sequence Models and Long-Short Term Memory Networks


At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding hidden state $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.






In [ ]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Prepare data:

In [ ]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("The cat pushes the ball".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Students like mathematics".split(), ["NN", "V", "NN"]),
    ("Deep learning is a machine learning tool".split(), ["NN", "NN", "V", "DET", "NN", "NN", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

Print the structure of the training data

In [ ]:
print(training_data)
print(training_data[0])
print(training_data[0][0])

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}



In [ ]:
ix_to_tag = {}
for i, tag in enumerate(tag_to_ix):
    ix_to_tag[i] = tag

print(ix_to_tag)

In [ ]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 10

## Play with the LSTM:



A few cells to understand and test the LSTM

In [ ]:
vocab_size = len(word_to_ix)
print(vocab_size)
embedding_dim = EMBEDDING_DIM
hidden_dim = HIDDEN_DIM
word_embeddings = nn.Embedding(vocab_size, embedding_dim)
print(word_embeddings)


In [ ]:
sentence = training_data[0][0]
print(sentence)
sentence_in = prepare_sequence(sentence, word_to_ix)
print(sentence_in)



### Question: how to interpret the content of "embeds" in the next cell?

#### Response: it is the random embedding of the sentence "sentence_in".

In [ ]:
embeds = word_embeddings(sentence_in)
print(embeds)
print(embeds.shape)

In [ ]:
hidden_state_dim = 4
rnn = nn.LSTM(10, hidden_state_dim) # input dim =10, hidden state dim = hidden_state_dim
input = torch.randn(5, 3, 10)
# 5: number of words per sentence,
# 3: number of sentences (batch size),
# 10 the embedding dimension of the word
#print(input)
h0 = torch.randn(1, 3, hidden_state_dim)
# 1: number of hidden layers in a single lstm unit
# 3: number of sentences,
# hidden state dim = 4
#print(h0)
c0 = torch.randn(1, 3, hidden_state_dim)
# 1: number of hidden layers in a single lstm unit
# 3: number of sentences,
# hidden state dim = 4
#print(c0)
output, (hn, cn) = rnn(input, (h0, c0))
#print(output)
print(output.shape)
# 5: number of words per sentence,
# 3: number of sentences,
# 4: hidden state returned by the lstm unit
print(hn.shape)
# 1: "last" hidden state in the sequence
# 3: number of sentences,
# 4: hidden state returned by the lstm unit
print(cn.shape)
# 1: "last" cell state in the sequence
# 3: number of sentences,
# 4: cell state returned by the lstm unit

print(output[:,2,:])
print(hn[0,2,:])
print(cn[0,2,:])

### Question: how to interpret the content of "lstm_out" in the next cell?

#### Response: it is the output from the last layer of the LSTM (see the next cell)

In [ ]:
embeds_view = embeds.view(len(sentence), 1, -1)
print(embeds_view)
print(embeds_view.shape)

lstm = nn.LSTM(embedding_dim, hidden_dim)
# Parameters of LSTM
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
lstm_out, _ = lstm(embeds_view)
# LSTM input of shape (seq_len, batch, input_size):
#tensor containing the features of the input sequence.
#
# LSTM output of shape (seq_len, batch, num_directions * hidden_size):
# tensor containing the output features (h_t) from the last layer of the LSTM, for each t.
print(lstm_out)
print(lstm_out.shape)

## Create the model:

### Question: add a LSTM layer in the following neural network with an embedding layer.

#### Response: Just look at the code.

### Question: what is the role of the linear layer in the following neural network?

#### Response: We add a linear layer for tagging the word (classification task).

### Question: how to interpret the output of the "forward" function?

#### Response: For each word, it gives the values of the possible tags. Hence, there are 3 values. The maximum value corresponds to the most likely tag.

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

## Train the model:



In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i
    return index

def get_max_prob_result(input, ix_to_tag):
    return ix_to_tag[get_index_of_max(input)]

### Question: compute and print the tags before the training for the input defined below.

#### Response: see the next cell.

In [ ]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(training_data[0][0])
    print(inputs)
    print(tag_scores)
    for i, tags in enumerate(tag_scores):
        print('Prediction: {}'.format(get_max_prob_result(tags, ix_to_tag)))


### Question: write the code that trains the neural network.

#### Response: see the next cell.

In [ ]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

### Question: compute and print the tags after the training for the input defined below.

#### Response: see the next cell.

In [ ]:
# See what the scores are after training
test_data = [
    ("The cat ate the book".split(), ["DET", "NN", "V", "DET", "NN"])
]

with torch.no_grad():
    inputs = prepare_sequence(test_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

    for i, tags in enumerate(tag_scores):
      print('Prediction: {}'.format(get_max_prob_result(tags, ix_to_tag)))
